# Bollinger Bands Strategy and Backtesting

Prepared by Samalie Piwan   
Email : spiwan@andrew.cmu.edu

## 1. Strategy Implementation

Load the libraries to be used in the project

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import yfinance as yf
import requests
from backtesting import Backtest, Strategy
from tabulate import tabulate
backtesting.set_bokeh_output(notebook=False)

Declare the Simple Moving Average Window  and Standard Deviation to run the first version of the strategy with

In [129]:
# Declare the variables that will be used globally as SMA window, Short Period and Long Period / Standard Deviation
sma_window = 20
std = 2

Set the fiat currency pair to run the first version of the strategy on

In [130]:
period = "1y"
backtest_start='2005-01-01'
backtest_end = '2022-12-31'

### 1.1 Backtesting Slice One of Dataset - Jan 2005 to Dec 2013

Define the Indicator(Bollinger Bands) and Buy / Sell Strategy that the backtesting.py library will optimize

In [131]:
def indicator(data, window, std_dev):
    bbands = ta.bbands(close = data.Close.s, length = window, std = std_dev)
    return bbands.to_numpy().T[:3]

class BollingerBandsStrategy(Strategy):
        
    sma_window = sma_window
    std = std
    
    def init(self):
        self.bbands = self.I(indicator, self.data, self.sma_window, self.std)
        
    def next(self):
        lower_band = self.bbands[0]
        upper_band = self.bbands[2]

        if self.position:
            if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
        else:
            if self.data.Close[-1] < lower_band[-1]:
                self.buy()

Create a list of the currency pairs

In [168]:
currency_pairs = ['EURUSD=X', 'USDJPY=X', 'GBPUSD=X', 'AUDUSD=X', 'USDCAD=X', 'USDCHF=X', 'EURGBP=X', 'EURJPY=X', 'GBPJPY=X', 'EURCHF=X', 'AUDJPY=X', 'USDCNY=X', 'USDMXN=X', 'USDHKD=X', 'USDSGD=X']

Load the data from the API

In [169]:
for forex_pair in currency_pairs:
    
    data = yf.download(forex_pair, start=backtest_start, end=backtest_end)

    #Seperate the data into two sets, 2005 to 2013 and 2014 to 2022
    backtest_dataset = pd.DataFrame(data.loc['2005-01-01' : '2013-12-31'].copy())
    rerun_dataset = pd.DataFrame(data.loc['2014-01-01':'2022-12-31'].copy())
    
    backtest_filename = "backtest"+forex_pair+".csv"
    rerun_filename = "rerun"+forex_pair+".csv"

    backtest_dataset.to_csv("backtestdata/"+backtest_filename)
    rerun_dataset.to_csv("rerundata/"+rerun_filename)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Define the function that will run the backtests for the currency pair list above

In [170]:
backtest_results = []
backtest_dir = 'backtestdata'
files = os.listdir(backtest_dir)

def backtest_strategy(param_cash, param_commission):
    index = 0
    while index < len(files):
        try:
            filename = backtest_dir+"/"+files[index]
            symbol = filename[21:27]
            
            file_data = pd.read_csv(filename)
            
            data_df = file_data.copy()
            data_df.index = pd.DatetimeIndex(data_df['Date'])
            data_df.sort_index(ascending=True, inplace=True)
            
            ohlc_data = data_df.resample("1H").apply({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'}).ffill()
           
            bt = Backtest(data_df, BollingerBandsStrategy, cash = param_cash, commission = param_commission)
            stats = bt.run()
            backtest_results.append(list([symbol, stats]))
        
        except FileNotFoundError:
            print(f"File {filename} not found")
        
        except ValueError:
            break
        
        finally:
            index += 1
            
    return backtest_results

In [172]:
bulk_backtest_results = backtest_strategy(10000, 0.01)

In [174]:
for results in bulk_backtest_results:
    print(results[0])
    stats = results[1]
    pnl_backtest = np.cumsum(stats['_trades'])
    pnl_backtest_val = pnl_backtest['PnL'].iloc[-1]
    print(pnl_backtest_val)

AUDJPY
-1211.9439897155723
AUDUSD
-1108.504819706688
EURCHF
-2357.9539690840265
EURGBP
-1829.6338996845495
EURJPY
-2355.8473608398394
EURUSD
-2397.3550437414638
GBPJPY
-3126.825851593023
GBPUSD
-2041.2635893452302
USDCAD
-2485.649643279318
USDCHF
-2466.3597611713362
USDCNY
-2997.941645092963
USDHKD
-2235.8260506153106
USDJPY
-1425.432366104128
USDMXN
-1456.5712540531124
USDSGD
-2986.3061991429313


### 2.2 Cross validation

From our previous backtesting results, the optimal paramters are:

 - Optimal standard deviation : 1.4
 - Optimal SMA Window : 29

In [85]:
optmized_std = 1.4
optimized_window = 29

Run an instance of our Backtest class using  from section 2 using:
- The **'rerun'** files that contain data from 2014 to 2022
- Cash of $10,000
- Commission at 1%

In [104]:
class OptBollingerBandsStrategy(Strategy):
    
    std = optmized_std
    sma_window = optimized_window
    
    def init(self):
        self.bbands = self.I(indicator, self.data, self.sma_window, self.std)
        
    def next(self):
        lower_band = self.bbands[0]
        upper_band = self.bbands[2]

        if self.position:
            if self.data.Close[-1] > upper_band[-1]:
                self.position.close()
        else:
            if self.data.Close[-1] < lower_band[-1]:
                self.buy()

In [117]:
rerun_results = []
rerun_dir = 'rerundata'
files = os.listdir(rerun_dir)

def validate_strategy(param_cash, param_commission):
    index = 0
    while index < len(files):
        try:
            filename = rerun_dir+"/"+files[index]
            symbol = filename[15:21]
            
            file_data = pd.read_csv(filename)
            
            data_df = file_data.copy()
            data_df.index = pd.DatetimeIndex(data_df['timestamp'])
            data_df.sort_index(ascending=True, inplace=True)
            
            bt = Backtest(data_df, OptBollingerBandsStrategy, cash = param_cash, commission = param_commission)
            stats = bt.run()
            rerun_results.append(list([symbol, stats]))
        
        except FileNotFoundError:
            print(f"File {filename} not found")
        
        except ValueError:
            break
        
        finally:
            index += 1
            
    return rerun_results

In [118]:
bulk_validate_results = validate_strategy(10000, 0.01)

In [120]:
for result in bulk_validate_results:
    print(result[0])
    stats = result[1]
    pnl_rerun = np.cumsum(stats['_trades'])
    pnl_rerun_val = pnl_rerun['PnL'].iloc[-1]
    print(pnl_rerun_val)

AUDCHF
-882.9188419999988
AUDJPY
570.0467199999985
AUDUSD
1121.5464528000007
CADJPY
952.6324799999995
EURAUD
1082.7606537000001
EURCHF
-1035.9924848
EURGBP
451.77775489999846
EURJPY
2322.2707600000003
EURUSD
502.15364440000064
GBPAUD
657.3007135999995
GBPCHF
-1319.978268
GBPJPY
696.6432000000002
GBPUSD
-597.2395260000004
NZDUSD
221.77492310000022
USDCAD
1550.7800684000026
USDCHF
1011.2709577999985
USDJPY
727.7078399999991
USDMXN
-98.9501399999993
USDSGD
602.4581310000017
